In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-31"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
4460,2024-05-31,México Cibacopa,00:15,Halcones de Obregon,Rayos de Hermosillo,2.05,1.70,178.5,1.85,1.83,1.5,1.91,1.56,xIu5ZE1n,0.487805,0.588235,0.540541,0.546448,0.076040,267.606,50.993598,0.190555,1.2,1.643168,1.369306,215.25,2.958,0.568040,0.192035,-18.0,136.148,23.624660,0.173522,1.8,1.643168,0.912871,161.50,1.480,0.167183,0.112961,28.0,105,95,2.05,1.70,173.468,170.406,0.131993,0.007686,0.142644,1.53,0.306,3.431373,0.576529,0.7,0.123471,0.36,0.072,9.722222,0.534888,0.5,-0.034888
4461,2024-05-31,Israel Liga Leumit,08:00,Elitzor Netania,Gilboa Galil,1.45,2.62,154.5,1.86,1.80,0.0,0.00,0.00,bXTP5dAM,0.689655,0.381679,0.537634,0.555556,0.071335,138.128,50.696336,0.367024,3.0,0.000000,0.000000,97.92,1.622,0.409902,0.252714,56.0,144.200,49.539851,0.343550,2.4,1.341641,0.559017,210.60,1.834,0.419976,0.228995,17.0,72,90,1.36,2.34,110.172,180.454,0.406543,0.023184,NaN,1.69,0.338,1.331361,0.714030,0.9,0.185970,5.77,1.154,1.403813,0.435989,0.7,0.264011
4462,2024-05-31,Israel Superliga,10:15,Nes Ziona,Maccabi Tel Aviv,4.51,1.20,171.5,1.87,1.79,0.0,0.00,0.00,dUvjGcLH,0.221729,0.833333,0.534759,0.558659,0.055063,158.214,44.177715,0.279228,2.4,1.341641,0.559017,126.84,1.894,0.498026,0.262949,44.0,118.032,14.945913,0.126626,2.4,1.341641,0.559017,126.36,1.320,0.240104,0.181897,38.0,84,117,1.51,1.08,126.682,118.932,0.819798,0.030912,NaN,-0.58,-0.116,-30.258621,0.618450,0.4,-0.218450,-0.05,-0.010,-20.000000,0.754096,0.8,0.045904
4463,2024-05-31,Israel Superliga,07:45,Hapoel Holon,Hapoel Jerusalem,1.69,2.15,159.5,1.87,1.87,-3.5,2.01,2.48,IykALhcc,0.591716,0.465116,0.534759,0.534759,0.056832,179.118,37.851308,0.211321,1.8,1.643168,0.912871,216.60,2.374,0.753346,0.317332,-26.0,139.878,24.782399,0.177172,2.4,1.341641,0.559017,166.46,1.610,0.286793,0.178132,49.0,76,82,2.85,2.03,129.038,160.828,0.169411,0.000000,0.148036,1.35,0.270,2.555556,0.648860,0.7,0.051140,1.14,0.228,5.043860,0.679685,0.7,0.020315
4464,2024-05-31,Indonésia Ibl,10:00,Satya Wacana,Pacific Caesar Surabaya,2.05,1.70,156.5,1.87,1.79,-4.5,1.85,2.28,YmX54fNb,0.487805,0.588235,0.534759,0.558659,0.076040,394.338,400.850972,1.016516,1.8,1.643168,0.912871,184.11,4.954,4.623043,0.933194,1.0,377.240,174.723049,0.463162,0.0,0.000000,NaN,318.72,4.978,2.457696,0.493712,-40.0,51,83,3.61,3.84,214.334,454.364,0.131993,0.030912,0.147243,-0.79,-0.158,-6.645570,0.000000,0.0,0.000000,-5.00,-1.000,-0.700000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4520,2024-05-31,Ásia Wasl,12:00,Kuwait SC,Sagesse Beirut,1.93,1.79,172.5,1.80,1.90,-3.5,1.95,2.27,b1gDvUUb,0.518135,0.558659,0.555556,0.526316,0.076794,146.554,45.100101,0.307737,2.4,1.341641,0.559017,106.92,1.556,0.480448,0.308771,62.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,99,103,1.08,1.17,110.820,0.000,0.053223,0.038222,0.107239,-1.50,-0.300,-3.100000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
4521,2024-05-31,Bélgica Liga Pro De Basquete,15:30,Oostende,Antwerp Giants,1.42,2.72,154.5,1.86,1.84,0.0,0.00,0.00,bBrjBKyI,0.704225,0.367647,0.537634,0.543478,0.071872,114

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
